### Setting environment variables

In [1]:
# Import the os module to interact with the operating system environment variables
import os


# Set the LANGCHAIN_PROJECT environment variable to specify the project name in LangChain
os.environ['LANGCHAIN_PROJECT'] = "agentic-workflow"

# Set the SERPAPI_API_KEY environment variable with your SERP API key
os.environ['SERPAPI_API_KEY'] = "b218c1eaa132eee166f5387e992dbdd3b05fc945e0f59809652d10a8c6a26d0b"

# Set the LANGCHAIN_TRACING_V2 environment variable to enable tracing for LangChain version 2
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Set the LANGCHAIN_API_KEY environment variable with your LangChain API key
os.environ["LANGCHAIN_API_KEY"] = "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"

# Set the OPENAI_API_KEY environment variable with your OpenAI API key
#os.environ["OPENAI_API_KEY"] = "a164a2381286481bac2afbbf622114fb" #"sk-proj-tKZzJhFV0n3RBNvcFiiYT3BlbkFJFXX6o83ZmMZ6gycFqesG"

os.environ["AZURE_OPENAI_API_KEY"] = "01a86ebdb2fc4d2ea1660672e161a116"
os.environ["AZURE_OPENAI_EMBEDDING_MODEL_NAME"] = 'nithin-text-embedding-ada-002'
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://mandarnithinpraveen.openai.azure.com/"

# Set the PINECONE_API_KEY environment variable with your Pinecone API key
os.environ["PINECONE_API_KEY"] = "9602b65f-cc2a-41c6-8a40-f56b499dc060"

# Set the LANGSMITH_USER_HANDLE environment variable with your Langsmith user handle
os.environ['LANGSMITH_USER_HANDLE'] = "earnings"

In [2]:
# Import the Client class from the langsmith module to create a client instance
from langsmith import Client

# Initialize a new client instance using the Client class
client = Client()

### Loading libraries

In [3]:
# Import the itemgetter function from the operator module to extract items from objects
from operator import itemgetter

# Import the hub module from langchain to access its functionalities
from langchain import hub

# Import the AgentExecutor class from langchain.agents to execute agent actions
from langchain.agents import AgentExecutor

# Import the load_tools function from langchain.agents to load necessary tools for agents
from langchain.agents import load_tools

# Import the format_to_openai_functions function from langchain.agents.format_scratchpad
# to convert scratchpad content into OpenAI functions
from langchain.agents.format_scratchpad import format_to_openai_functions

# Import the OpenAIFunctionsAgentOutputParser class from langchain.agents.output_parsers
# to parse the output of agents into OpenAI functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

# Import the ChatOpenAI class from langchain.chat_models to interact with OpenAI chat models
from langchain.chat_models import ChatOpenAI,AzureChatOpenAI

# Import the format_tool_to_openai_function function from langchain.tools.render
# to format tools into OpenAI functions
from langchain.tools.render import format_tool_to_openai_function

In [4]:
#embeddings = AzureOpenAIEmbeddings(azure_endpoint="https://mandarnithinpraveen.openai.azure.com/",
                              #openai_api_key="01a86ebdb2fc4d2ea1660672e161a116", model='nithin-text-embedding-ada-002')
llm = AzureChatOpenAI(model="nithin-gpt35-turbo", temperature=0,api_version="2024-02-01")

c:\program files\python38\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [6]:
!pip install numexpr

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/95.0 kB ? eta -:--:--
   -------------------------------------- - 92.2/95.0 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 95.0/95.0 kB 1.4 MB/s eta 0:00:00


### Initializing LLMs and Tools

In [5]:
# Create an instance of the ChatOpenAI class to interact with OpenAI's chat models
#llm = ChatOpenAI()

# Load tools such as serpapi, wikipedia, and llm-math using the load_tools function
tools = load_tools(["serpapi", "wikipedia", "llm-math"], llm=llm)

# Bind the loaded tools to the language model (llm) by converting them into OpenAI functions
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

c:\program files\python38\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


### Prompt design

In [6]:
# Use the hub.pull method from the langchain hub to retrieve a specific prompt
prompt = hub.pull("wfh/agent-lcel-prompt")

In [27]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'wfh', 'lc_hub_repo': 'agent-lcel-prompt', 'lc_hub_commit_hash': '08dfba0e76aaf99490887fca749eedca2541c34fcde5a21df5bf8b2960938e01'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are helpful AI assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')])

In [7]:
# Define a function named format_scratchpad that takes a dictionary as input
# The dictionary is expected to contain a key "intermediate_steps" whose value is processed
# using the format_to_openai_functions function to convert intermediate steps into a string
def format_scratchpad(x: dict) -> str:
    return format_to_openai_functions(x["intermediate_steps"])

# Define a function named get_chat_history that takes a list as input
# The list is expected to contain a dictionary with a key "chat_history"
# If the "chat_history" key exists, its value is returned; otherwise, an empty list is returned
def get_chat_history(x: list) -> str:
    return x.get("chat_history", [])

### Creating agents

In [22]:
# Define an agent configuration dictionary with specific functions mapped to keys
# "input" is mapped to the itemgetter function to extract the "input" field from a given object
# "agent_scratchpad" is mapped to the format_scratchpad function to process intermediate steps
# "chat_history" is mapped to the get_chat_history function to retrieve chat history
agent_con = {
    "input": itemgetter("input"),  # Map "input" to the itemgetter function
    "agent_scratchpad": format_scratchpad,  # Map "agent_scratchpad" to the format_scratchpad function
    "chat_history": get_chat_history  # Map "chat_history" to the get_chat_history function
}

# Apply the agent configuration to a prompt using the pipe operator
# This involves pulling the prompt from the hub, binding it with the loaded tools (llm_with_tools),
# and then processing it through the agent configuration
# The result is then parsed by the OpenAIFunctionsAgentOutputParser to format the output
agent = (
    agent_con
    | prompt
    | llm_with_tools  # Bind with loaded tools
    | OpenAIFunctionsAgentOutputParser()
)

#agent = OpenAIFunctionsAgentOutputParser.parse(agent)

In [28]:
agent

{
  input: RunnableLambda(itemgetter('input')),
  agent_scratchpad: RunnableLambda(format_scratchpad),
  chat_history: RunnableLambda(get_chat_history)
}
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'wfh', 'lc_hub_repo': 'agent-lcel-prompt', 'lc_hub_commit_hash': '08dfba0e76aaf99490887fca749eedca2541c34fcde5a21df5bf8b2960938e01'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are helpful AI assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')])
| RunnableBinding

In [23]:
# Instantiate an AgentExecutor object with the specified agent configuration and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### Questions to agents

In [24]:
inputs = ["who is Indian prime minister",
          "What is US President's age and return the number age multiplied by two",
          "what is the name of OpenAI's new AI model?"]

In [25]:
results = agent_executor.batch([{"input": x} for x in inputs], return_exceptions=True)



> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Indian prime minister`



Invoking: `Search` with `current US President age`


81 yearsOpenAI's new AI model is called GPT-4o. It was recently launched, and you can find more information about it on CNBC, CNN, and CBS News.

> Finished chain.
The current US President's age is 81. When multiplied by 2, the result is 162.

> Finished chain.
Page: Prime Minister of India
Summary: The prime minister of India (IAST: Bhārat kē Pradhānamantrī) is the head of government of the Republic of India. Executive authority is vested in the prime minister and his chosen Council of Ministers, despite the president of India being the nominal head of the executive.  The prime minister has to be a member of one of the houses of bicameral Parliament of India, alongside heading the respective house. The prime minister and their cabinet are at all times responsib

In [26]:
results

[{'input': 'who is Indian prime minister',
  'output': 'The current Prime Minister of India is Narendra Modi, who has been in office since 26 May 2014.'},
 {'input': "What is US President's age and return the number age multiplied by two",
  'output': "The current US President's age is 81. When multiplied by 2, the result is 162."},
 {'input': "what is the name of OpenAI's new AI model?",
  'output': "OpenAI's new AI model is called GPT-4o. It was recently launched, and you can find more information about it on CNBC, CNN, and CBS News."}]